In [20]:
#!pip install hangul-utils
!pip install --upgrade gensim
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import pandas as pd
import numpy as np
import re
from soynlp.noun import LRNounExtractor_v2
from soynlp.tokenizer import LTokenizer
from sklearn.metrics.pairwise import cosine_similarity
#from hangul_utils import split_syllables

In [23]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/LIFLOW/data/AI_chatBot.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/LIFLOW/data/AI_speaker.csv')
df3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/LIFLOW/data/studio.csv')
df4 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/LIFLOW/data/voice_collection_tool.csv')

df = pd.concat([df1, df2, df3, df4])
df.reset_index(drop = True, inplace = True)
df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,sentence,Id,fileNm,recrdTime,recrdQuality,recrdDt,scriptSetNo,recrdEnvrn,colctUnitCode,cityCode,recrdUnit,convrsThema,gender,recorderId,age
0,0,밥 한끼를 제대로 된 밥 한끼를 먹을 수 있다고 생각하면서,노인대화-08580,노인남여_노인대화07_F_1522434093_60_경상_실내_08580.wav,4.52,16K,2020-11-21 20:44:24,T_노인대화_7,실내,AI 챗봇,경상,AndroidOS,방송/연예,여,1522434093,60
1,1,감사하게 감사하게 생각하면서,노인대화-08581,노인남여_노인대화07_F_1522434093_60_경상_실내_08581.wav,3.67,16K,2020-11-21 20:44:31,T_노인대화_7,실내,AI 챗봇,경상,AndroidOS,방송/연예,여,1522434093,60
2,2,노후를 같이 보내야 되지 않을까 싶네요,노인대화-08582,노인남여_노인대화07_F_1522434093_60_경상_실내_08582.wav,3.58,16K,2020-11-21 20:44:38,T_노인대화_7,실내,AI 챗봇,경상,AndroidOS,방송/연예,여,1522434093,60
3,3,제가 가장 좋아하는 연예인은 별에서 온 그대에 전지현이고,노인대화-08583,노인남여_노인대화07_F_1522434093_60_경상_실내_08583.wav,4.52,16K,2020-11-21 20:44:46,T_노인대화_7,실내,AI 챗봇,경상,AndroidOS,방송/연예,여,1522434093,60
4,4,잔디씨가 좋아하는 연예인은 누구에요,노인대화-08584,노인남여_노인대화07_F_1522434093_60_경상_실내_08584.wav,3.41,16K,2020-11-21 20:44:53,T_노인대화_7,실내,AI 챗봇,경상,AndroidOS,방송/연예,여,1522434093,60


In [24]:
# 띄어쓰기 등 제거
df['convrsThema'] = df['convrsThema'].str.replace(' ', '')

In [25]:
convrsThemas = df['convrsThema'].unique()

In [26]:
convrsThemas

array(['방송/연예', '경제/재태크', 'PBS', '휴일', '먹거리', '건강/다이어트', '국가/지역', '자동차',
       '영화', '스포츠/레저', '연애/결혼', '문학', '자연/휴양지', '양육/교육', '시사/재테크',
       '자연/여행', '건강/교육', '여행/콘서트', '날씨/일상', '여행/건강', '여행/자녀', 'TV',
       '먹거리/건강', '#일상', '미래', '좋은글,사자성어,좋은글귀,등등읽으면도움이되는글들..', '반려동물,강아지',
       '대화', '말씀나누기', '아무거나', '#영화#여행#독서#취미#교육', '취미생활', '혼자', '일상이야기',
       '#여행', '일반', '게읻트볼', '#인생', '직업', '일상대화', nan,
       '여행,지인,집,축구,음식점,자녀등',
       '드라마,영화,중국음식,건강,쇼핑,연예인,영어,라섹,두발자유,학생화장,항공,은행,노트북,커피,운동등', '군대',
       '야구,축구,지하철역,집값,정치,기차등', '건강,반려견,자녀입대,아르바이트,여행,부모님,콘서트,여가생활등',
       '음식,핸드폰,게임,가족,회사,영양제등', '코로나/생활', '게임', '만화', '결혼', '가정', '건강',
       '일상생활', '휴양지', '운동', '여행', '드라마', '연예인', '골프', '여가생활', '가족',
       '결혼생활', '정신건강', '가족건강', '식습관', '피시방', '부모님', '청소', '자녀결혼', '해외여행',
       '친구', '볼링', '건강식품', '재테크', '학창시절', '일화', '운전면허', '다이어트', '캠핑',
       '투병', '환경', '일본', '육아', '전자기기', '음식점', '음식', '임신', '일', '연애',
       '시집살이', '국제관계', '경제', '노후', '돈', '자녀교육', '자녀', '일상', '책', '야구',


In [27]:
convrsThema_count = dict([])
for i in range(len(df)) :
  thema = df.loc[i, 'convrsThema']
  if thema in convrsThema_count :
    convrsThema_count[thema] += 1
  else :
    convrsThema_count[thema] = 1
convrsThema_count

{'방송/연예': 21792,
 '경제/재태크': 78342,
 'PBS': 23502,
 '휴일': 42975,
 '먹거리': 104023,
 '건강/다이어트': 151226,
 '국가/지역': 62755,
 '자동차': 57091,
 '영화': 47405,
 '스포츠/레저': 21870,
 '연애/결혼': 16886,
 '문학': 48088,
 '자연/휴양지': 55530,
 '양육/교육': 25109,
 '시사/재테크': 23259,
 '자연/여행': 5163,
 '건강/교육': 12144,
 '여행/콘서트': 9423,
 '날씨/일상': 2320,
 '여행/건강': 7706,
 '여행/자녀': 6692,
 'TV': 643,
 '먹거리/건강': 2855,
 '#일상': 165,
 '미래': 11,
 '좋은글,사자성어,좋은글귀,등등읽으면도움이되는글들..': 110,
 '반려동물,강아지': 140,
 '대화': 1,
 '말씀나누기': 1447,
 '아무거나': 2198,
 '#영화#여행#독서#취미#교육': 608,
 '취미생활': 503,
 '혼자': 956,
 '일상이야기': 1154,
 '#여행': 1,
 '일반': 810,
 '게읻트볼': 1143,
 '#인생': 58,
 '직업': 12096,
 '일상대화': 1000,
 nan: 1000,
 '여행,지인,집,축구,음식점,자녀등': 2558,
 '드라마,영화,중국음식,건강,쇼핑,연예인,영어,라섹,두발자유,학생화장,항공,은행,노트북,커피,운동등': 6623,
 '군대': 4040,
 '야구,축구,지하철역,집값,정치,기차등': 3510,
 '건강,반려견,자녀입대,아르바이트,여행,부모님,콘서트,여가생활등': 2770,
 '음식,핸드폰,게임,가족,회사,영양제등': 4629,
 '코로나/생활': 485,
 '게임': 7842,
 '만화': 5496,
 '결혼': 2347,
 '가정': 2318,
 '건강': 18712,
 '일상생활': 7585,
 '휴양지': 7481,
 '운동': 19707,
 '여행': 

In [28]:
convrsThema_count_desc = sorted(convrsThema_count.items(), key = lambda x : x[1], reverse = True)

In [29]:
convrsThema_count_desc

[('건강/다이어트', 151226),
 ('먹거리', 104023),
 ('경제/재태크', 78342),
 ('국가/지역', 62755),
 ('자동차', 57091),
 ('자연/휴양지', 55530),
 ('문학', 48088),
 ('영화', 47405),
 ('휴일', 42975),
 ('양육/교육', 25109),
 ('PBS', 23502),
 ('시사/재테크', 23259),
 ('스포츠/레저', 21870),
 ('방송/연예', 21792),
 ('여행', 20043),
 ('운동', 19707),
 ('건강', 18712),
 ('일화', 18530),
 ('연애/결혼', 16886),
 ('건강/교육', 12144),
 ('직업', 12096),
 ('자녀결혼', 9544),
 ('여행/콘서트', 9423),
 ('경제', 8637),
 ('골프', 7883),
 ('게임', 7842),
 ('여행/건강', 7706),
 ('일상생활', 7585),
 ('휴양지', 7481),
 ('야구', 6886),
 ('여행/자녀', 6692),
 ('드라마,영화,중국음식,건강,쇼핑,연예인,영어,라섹,두발자유,학생화장,항공,은행,노트북,커피,운동등', 6623),
 ('책', 5926),
 ('만화', 5496),
 ('연예인', 5409),
 ('자연/여행', 5163),
 ('음식', 5110),
 ('자녀교육', 5009),
 ('노후준비', 4822),
 ('음식,핸드폰,게임,가족,회사,영양제등', 4629),
 ('운전', 4558),
 ('해외여행', 4328),
 ('자녀', 4152),
 ('스포츠', 4118),
 ('군대', 4040),
 ('재테크', 4016),
 ('야구,축구,지하철역,집값,정치,기차등', 3510),
 ('여가생활', 3364),
 ('음식점', 3048),
 ('집', 2959),
 ('교육', 2922),
 ('먹거리/건강', 2855),
 ('건강,반려견,자녀입대,아르바이트,여행,부모님,콘서트,여가생활등'

In [30]:
sentences = []
cityCode = []
convrsThema = []
gender = []
age = []

tmp_convrsThema = '방송/연예'
tmp_sentence = ''
for i in range(len(df)) :
    if df.loc[i, 'convrsThema'] != tmp_convrsThema :
        sentences.append(tmp_sentence)
        convrsThema.append(tmp_convrsThema)
        gender.append(df.loc[i-1, 'gender'])
        age.append(df.loc[i-1, 'age'])
        cityCode.append(df.loc[i-1, 'cityCode'])
        tmp_sentence = ''
        tmp_convrsThema = df.loc[i, 'convrsThema']
    tmp_sentence += df.loc[i, 'sentence'] + ' '

In [31]:
main_df = pd.DataFrame({'sentence' : sentences, 'cityCode' : cityCode, 'convrsThema' : convrsThema, 'gender' : gender, 'age' : age})
main_df['convrsThema'].fillna('nan', inplace=True)

In [32]:
main_df['convrsThema'] = main_df['convrsThema'].str.replace('코로나/생활', '투병')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('메르스', '투병')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('축구선수', '축구')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 문학 ', '책')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('독서', '책')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('문학', '책')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('집값', '부동산')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('분양', '부동산')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('직업', '직장')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('일', '직장')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('좋은글,사자성어,좋은글귀,등등 읽으면 도움이되는 글들..', '좋은글')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('국제관계', '정치')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('제테크', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('시사/재테크', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 경제/재테크 ', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 경제/재태크 ', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 자연/휴양지 ', '자연/여행')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자동차보험', '교통사고/보험')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('교통사고', '교통사고/보험')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('운전', '자동차')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 자동차 ', '자동차')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀취업', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀직장', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀입대', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀교육', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀결혼', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀 취업', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀 교육', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀 결혼', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('일상생활', '일상')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('일상 이야기', '일상')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('일상대화', '일상')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('#일상', '일상')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('이사', '이사/이민')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('이민', '이사/이민')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('먹거리', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('먹거리/건강', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 먹거리 ', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('육아', '육아')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('양육/교육', '육아')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('교육', '교육')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('운동', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 스포츠/레저 ', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('영화관', '영화관')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 영화 ', '영화')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('연예인', '방송/연예')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 방송/연예 ', '방송/연예')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 연애/결혼 ', '연애')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('여행계획', '여행')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('여가생활', '여가')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('시댁살이', '시집살이')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('시댁', '시집살이')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('말씀나누기', '대화')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 만화 ', '만화')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('결혼생활', '결혼')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('남편', '결혼')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('건강/다이어트', '다이어트')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 건강/다이어트 ', '다이어트')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('노후준비', '노후')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('노후대책', '노후')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('넷플릭스', '방송')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('TV', '방송')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('PBS', '방송')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('드라마', '방송')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 군대 ', '군대')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('건강/교육', '건강')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('가족건강', '건강')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('가정', '가족')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('#인생', '인생')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('반려동물,강아지', '반려동물')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 국가/지역 ', '국가/지역')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 게임 ', '게임')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('건강식', '건강식품')

main_df['convrsThema'] = main_df['convrsThema'].str.replace('수영', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('볼링', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('축구', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('야구', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('배구', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('골프', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('직장상', '직장')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('#직장상', '직장')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('직장화', '직장')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('피시방', '게임')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('요리', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('밥', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('건강식품품', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('건강식품', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('식습관', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('음식점', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('음식/건강', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('영양제', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('제주도', '여행')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('휴양지', '여행')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('등산', '취미생활')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('캠핑', '취미생활')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('여가', '취미생활')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('게읻트볼', '취미생활')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자전거', '취미생활')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('스쿠버다이빙', '취미생활')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('해외여행', '여행')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('건강검진', '건강')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('연애/결혼', '연애')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('육아', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('스포츠/레저', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('재태크', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('부동산', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('경제', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('아파트', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('재개발', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('돈', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('경제/재테크', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('방송', '방송/연예')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('영화', '방송/연예')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('영화관', '방송/연예')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('책', '교육')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('교통사고/보험/보험', '교통사고/보험')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자동차면허', '자동차')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('벌초', '제사')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('장례', '제사')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('학창시절', '친구')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('스포츠/레저', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('여행/건강', '여행')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('여행/자녀', '자녀')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()


In [33]:
convrsThemas_unique = list(set(main_df['convrsThema'].tolist()))
len(convrsThemas_unique)

85

In [34]:
doc_idx_thema = dict([])
for i in convrsThemas_unique :
  tmp = []
  for j in range(len(main_df)) :
    if main_df.loc[j, 'convrsThema'] == i :
      tmp.append(j)
  doc_idx_thema[i] = tmp

In [35]:
min_convrsThema = ''
min = 100000
for i in doc_idx_thema :
  if len(doc_idx_thema[i]) < min :
    min = len(doc_idx_thema[i])
    min_convrsThema = i
print(f"최소 길이 주제 : {min_convrsThema}, {min}")

최소 길이 주제 : 직장반, 1


In [36]:
sentences = main_df['sentence']
sentences = sentences.str.replace('\([^)]*\)', '') # 괄호안문자제거
sentences = sentences.str.replace('[^ㄱ-힣 ]', '') # 한글 제외하곤 다 제거

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
main_df['sentence'] = sentences

In [38]:
sentences = list(main_df['sentence'])
noun_extractor = LRNounExtractor_v2()
noun_score = noun_extractor.train_extract(sentences)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 148449 from 14698 sents. mem=1.893 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=10421810, mem=1.959 Gb
[Noun Extractor] batch prediction was completed for 39394 words
[Noun Extractor] checked compounds. discovered 11958 compounds
[Noun Extractor] postprocessing detaching_features : 24947 -> 18917
[Noun Extractor] postprocessing ignore_features : 18917 -> 18712
[Noun Extractor] postprocessing ignore_NJ : 18712 -> 17974
[Noun Extractor] 17974 nouns (11958 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=1.999 Gb                    
[Noun Extractor] 54.65 % eojeols are covered


In [39]:
my_score = dict([])
for w, s in noun_score.items() :
  my_score[w] = noun_score[w].score

In [40]:
doc_according_thema = dict([])
for i in convrsThemas_unique :
  sentences_thema = list((main_df[main_df['convrsThema'] == i])['sentence'])
  len_ = len(sentences_thema)
  if len_ < 10 :
    sentences_thema = sentences_thema[:len_]
  else :
    sentences_thema = sentences_thema[:10]
  sentences_thema_str = ' '.join(sentences_thema[:10])
  doc_according_thema[i] = sentences_thema_str

In [41]:
# 불용어 불러오기
with open('/content/drive/MyDrive/Colab Notebooks/LIFLOW/data/stop_words_0_100.txt') as f :
  stop_words = f.readlines()
stop_words = [s[:-1] for s in stop_words]

# 토크나이징 하기
tokens_according_thema = dict([])
tokenizer = LTokenizer(scores = my_score)
for i in doc_according_thema :
  tokens = tokenizer.tokenize(doc_according_thema[i])
  tmp_tokens = []
  for j in tokens :
    if j not in stop_words :
      tmp_tokens.append(j)
  tokens_according_thema[i] = tmp_tokens

In [42]:
# keyBERT로 키워드 추출하기
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

keyword_according_thema = dict([])
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
for n, i in enumerate(doc_according_thema) :
  print(n)
  doc = doc_according_thema[i]
  tokenized_nouns = ' '.join(tokens_according_thema[i])
  #n_gram_range = (2, 3)
  count = CountVectorizer().fit([tokenized_nouns])
  candidates = count.get_feature_names_out()

  doc_embedding = model.encode([doc])
  candidate_embeddings = model.encode(candidates)

  top_n = 10
  distances = cosine_similarity(doc_embedding, candidate_embeddings)
  keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
  keyword_according_thema[i] = keywords
  

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84


In [43]:
from gensim import models

ko_model = models.fasttext.load_facebook_model('/content/drive/MyDrive/Colab Notebooks/FastText_pretrained/cc.ko.300.bin')

ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xa1\x9c'; replacing invalid characters, using '\\xed\\xa1\\x9c'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xb0\x80'; replacing invalid characters, using '\\xed\\xb0\\x80'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xa7\x80'; replacing invalid characters, using '\\xed\\xa7\\x80'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xb8\xb0'; replacing invalid characters, using '\\xed\\xb8\\xb0'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xa0\x9c'; replacing invalid characters, using '\\xed\\xa0\\x9c'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xa6\xac'; replacing invalid characters, using '\\xed\\xa6\\xac'
ERROR:gensim.models._fasttext_bin:failed to decode invalid unicode bytes b'\xed\xb3\xb5'; replacing invalid characters, using '\\xed\\xb

In [44]:
embedding_according_thema = dict([])
for i in keyword_according_thema :
  base = ko_model.wv[keyword_according_thema[i][0]].copy()
  for j in range(1, len(keyword_according_thema[i])) :
    base += ko_model.wv[keyword_according_thema[i][j]].copy()
  embedding_according_thema[i] = base / 10

In [45]:
for i in keyword_according_thema :
  print(f"{i} : {keyword_according_thema[i]}")

취미생활 : ['삼학년', '버가모', '떡밥', '백록담', '배불리', '마하랄레는', '사투리', '간청', '마하라', '케냐는']
대화 : ['일하면', '루축시켜', '일하면서', '했답니다', '일하시오', '만나게', '질려냈습니다', '괴로워하고', '했잖아요', '대속하시려고']
게임 : ['배울', '중고등학생들', '학생', '중학교때', '학생들', '고등학교때', '대학생', '게임', '게임중독', '게임때문']
휴직장 : ['해야죠', '해봐야지', '힘들죠', '알려주잖아', '배웠어야하는데', '싶다든지', '힘들었으니까', '놀러가잖아', '안따라주니까요', '안따라줄까요']
지역 : ['성도까지', '문화', '밤까지', '맞춰서', '시켜서', '따라가면', '피난처였거든요', '걸어갔거든요', '똑같잖아요', '길이니까']
직장출 : ['산인가', '몰랐거든', '해돋', '충격적', '관광', '경험', '영인산', '산들이', '겨울', '비가']
스포츠,스포츠,지하철역,재테크,정치,기차등 : ['홍보', '자영업', '힘들더라고', '힘들었는데', '초보자', '프로야구', '힘들지', '꼴등하는데', '서울', '김광현']
전자기기 : ['살만하고', '읽는', '읽어봐야', '문학', '소설', '영어', '아마존', '소설들', '소설책', '영어책']
부산 : ['좋았잖아', '먹어서', '넘게', '오랜만', '좋았다고', '했거든', '신비', '부산', '저렴', '이십년']
자녀 : ['다른여자', '애로사항', '쇼핑', '해주셨잖니', '언니', '여자애들', '학원', '학교', '부모', '공부']
보험 : ['연구', '위협', '붙는다고', '붙이니', '붙지도', '힘들다고', '말하더라고요', '엮으니까', '비싸', '받는다고']
교육 : ['책마다', '책을', '소설', '역사소설', '문학적', '작가들', '문학', '책들', '소설들', '소설책']
가족 : ['말했잖아', '

In [46]:
cosine_similarity([ko_model.wv['어벤저스']], [ko_model.wv['토르']])

array([[0.32464686]], dtype=float32)

In [47]:
cosine_similarity([ko_model.wv['어벤저스']], [ko_model.wv['사과']])[0][0]

0.01915556

In [48]:
highsim_according_thema = dict([])
for i in keyword_according_thema :
  tmp = dict([])
  for j in keyword_according_thema :
    tmp[j] = cosine_similarity([embedding_according_thema[i]], [embedding_according_thema[j]])[0][0]
  tmp = sorted(tmp.items(), key = lambda x : x[1], reverse = True)
  highsim_according_thema[i] = tmp[1:6]

In [49]:
highsim_according_thema['운동선수']

KeyError: ignored

In [50]:
highsim_according_thema

{'취미생활': [('미래', 0.58796525),
  ('직장이야기', 0.5652983),
  ('임신', 0.55744845),
  ('직장출', 0.54362565),
  ('인간관계', 0.5336275)],
 '대화': [('가족', 0.6159253),
  ('휴직장', 0.60210556),
  ('군대', 0.5575584),
  ('직장', 0.5560809),
  ('지역', 0.5386796)],
 '게임': [('스포츠', 0.63273454),
  ('취업', 0.6271096),
  ('자녀', 0.606024),
  ('직장본', 0.58472013),
  ('부모님', 0.48483178)],
 '휴직장': [('직장', 0.6854975),
  ('가족', 0.65345615),
  ('군대', 0.63567436),
  ('대화', 0.60210556),
  ('자동차', 0.5841764)],
 '지역': [('직장', 0.549362),
  ('대화', 0.5386796),
  ('직장본', 0.52920896),
  ('휴직장', 0.5291479),
  ('군대', 0.5257161)],
 '직장출': [('#방송/연예#여행#교육#취미#교육', 0.70070046),
  ('국가/지역', 0.6837931),
  ('nan', 0.6740496),
  ('인생', 0.6710921),
  ('혼자', 0.57925636)],
 '스포츠,스포츠,지하철역,재테크,정치,기차등': [('교통', 0.5851056),
  ('지인', 0.5747818),
  ('당뇨', 0.5422629),
  ('취업', 0.53350693),
  ('자녀', 0.5144142)],
 '전자기기': [('교육', 0.8711315),
  ('만화', 0.60401917),
  ('방송/연예/연예', 0.53891),
  ('자녀', 0.52057916),
  ('이사/이민', 0.50328535)],
 '부산': [('이사/이사/이민', 0